In [1]:
# !conda install -c conda-forge fbprophet -y

In [2]:
# !pip install --upgrade plotly

In [3]:
from fbprophet import Prophet
import plotly.offline as py
# import pandas_datareader.data as web
import datetime
import pandas as pd

# import dash
# import dash_bootstrap_components as dbc
# import dash_html_components as html
# import dash_core_components as dcc
# import plotly.express as px
import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# from dash.dependencies import Input, Output,State

In [4]:
df=pd.read_csv("../stock.csv")
df['year_month']=pd.to_datetime(df['Date']).dt.strftime('%Y-%m')

In [5]:
df.head()

,Date,Symbols,Close,High,Low,Open,Volume,year_month
0,2020-12-03,AMZN,3186.73,3228.64,3181.3100,3205.46,2892039,2020-12
1,2020-12-03,GOOGL,1821.84,1843.83,1817.0000,1820.54,1236366,2020-12
2,2020-12-03,FB,281.85,286.65,281.0700,286.25,12921694,2020-12
3,2020-12-02,AMZN,3203.53,3232.00,3173.2601,3221.65,3129304,2020-12
4,2020-12-02,GOOGL,1824.97,1832.74,1785.1695,1795.36,1471235,2020-12


In [6]:
# df=df[['Date','High']]

In [7]:
# df = df.rename(columns={'Date': 'ds', 'High': 'y'})

In [8]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [9]:
df['High'] = pd.to_numeric(df['High'],errors='ignore')

In [10]:
df.shape

(702, 8)

In [11]:
df=df[df['Symbols']=='AMZN']
df=df[['Date','High']]
df = df.rename(columns={'Date': 'ds', 'High': 'y'})

In [12]:
df = df[df['ds']>='2020-01-02']
df0=df.copy()

In [13]:
df0.head()

,ds,y
0,2020-12-03,3228.64
3,2020-12-02,3232.00
6,2020-12-01,3248.95
9,2020-11-30,3228.39
12,2020-11-27,3216.19


In [14]:
estimated_days=180

In [15]:
df = df[:-estimated_days]

In [16]:
df.head()

,ds,y
0,2020-12-03,3228.64
3,2020-12-02,3232.00
6,2020-12-01,3248.95
9,2020-11-30,3228.39
12,2020-11-27,3216.19


In [17]:
df_model = Prophet(changepoint_prior_scale=0.5,yearly_seasonality=True,daily_seasonality=True)
df_model.fit(df)

C:\Users\soongaya\Anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



In [18]:
df_forecast = df_model.make_future_dataframe(periods= estimated_days, freq='D')

In [19]:
df_forecast = df_model.predict(df_forecast)

In [20]:
# df_model.plot_components(df_forecast)

# # Draw forecast results
# df_model.plot(df_forecast, xlabel = 'Date', ylabel = 'Stock')

# # Combine all graps in the same page
# plt.title(f'{estimated_days} Stocks')
# plt.title('STock')
# plt.ylabel('Stock')

# plt.show()

In [23]:
actual_stock = go.Scatter(
    name = 'Actual Stock',
   mode = 'lines',
   x = list(df0['ds']),
   y = list(df0['y']),
   marker=dict(
      color='black',
      line=dict(width=2)
   )
)

trend = go.Scatter(
    name = 'trend',
    mode = 'lines',
    x = list(df_forecast['ds']),
    y = list(df_forecast['yhat']),
    marker=dict(
        color='orange',
        line=dict(width=3)
    )
)


upper_band = go.Scatter(
    name = 'upper band',
    mode = 'lines',
    x = list(df_forecast['ds']),
    y = list(df_forecast['yhat_upper']),
    line= dict(color='green'),
    fill = 'tonexty'
)

lower_band = go.Scatter(
    name= 'lower band',
    mode = 'lines',
    x = list(df_forecast['ds']),
    y = list(df_forecast['yhat_lower']),
    line= dict(color='blue')
)


plot_data = [trend, lower_band, upper_band,actual_stock]

layout = dict(title='Stock Forecasting',
             xaxis=dict(title = 'Dates', ticklen=2, zeroline=False))

figure=dict(data=plot_data,layout=layout)

In [24]:
py.offline.iplot(figure)